# Scratch code for testing purposes and small experiments. 


In [2]:


import numpy as np
import importlib
from helpers.Graph import Graph
import pandas as pd
import random
from tqdm import tqdm

%load_ext autoreload
%autoreload 2


def common_neighbors(adj):
    return np.dot(adj, adj)


def eigenvalues(adj):
    return np.linalg.eigvals(adj)

def eigenvectors(adj):
    return np.linalg.eig(adj)




m1 = np.array([[0, 1, 0, 1, 0, 1, 0, 1, 0, 0],
                    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 1, 0, 1, 0, 0, 1],
                    [1, 1, 0, 0, 0, 1, 0, 1, 0, 0],
                    [0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
                    [1, 0, 0, 1, 0, 0, 0, 0, 1, 1],
                    [0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
                    [1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
                    [0, 0, 1, 0, 1, 1, 0, 0, 1, 0]])


m2 = np.array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
                    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
                    [1, 1, 0, 0, 0, 1, 0, 1, 0, 0],
                    [0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
                    [0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
                    [1, 0, 0, 0, 1, 0, 0, 0, 1, 1],
                    [1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
                    [0, 0, 1, 0, 1, 0, 1, 0, 1, 0]])





## Eigen values of G = G1 U G2 equal to the union of eigen values of G1 and G2 ?

In [15]:
eig_g1 = eigenvalues(m1)
eig_g2 = eigenvalues(m2)

m = np.logical_or(m1, m2).astype(int)

print("Eigen values of G1", eig_g1)
print("Eigen values of G2", eig_g2)
print("Eigen values of G", np.linalg.eigvals(m))



Eigen values of G1 [ 4.         -2.56155281 -2.23606798  2.23606798  1.56155281 -1.
 -1.          1.         -1.         -1.        ]
Eigen values of G2 [ 4.         -2.56155281 -2.23606798 -1.          2.23606798  1.
  1.56155281 -1.         -1.         -1.        ]
Eigen values of G [ 4.90768271  2.17008649  1.56914969 -2.83573973  0.31110782 -1.4811943
 -0.64109268 -2.         -1.         -1.        ]


Common neighbors matrix

In [16]:
A = common_neighbors(m) # or np.dot(m, m) for m^2
print("A = ", A)

A =  [[5 1 3 3 2 2 1 3 3 2]
 [1 4 1 1 1 3 3 2 0 2]
 [3 1 5 2 3 1 2 2 3 3]
 [3 1 2 4 1 2 2 2 2 1]
 [2 1 3 1 4 2 2 1 2 2]
 [2 3 1 2 2 6 5 3 2 2]
 [1 3 2 2 2 5 6 2 2 3]
 [3 2 2 2 1 3 2 5 2 3]
 [3 0 3 2 2 2 2 2 4 2]
 [2 2 3 1 2 2 3 3 2 5]]


In [17]:
m1 = np.array([     [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0 , 1], #0
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1 , 0], #1
                    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0 , 0], #2
                    [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0 , 0], #3
                    [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0 , 0], #4
                    [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0 , 0], #5
                    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0 , 0], #6
                    [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0 , 0], #7
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0 , 1], #8
                    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0 , 0], #9
                    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0 , 1], #10
                    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1 , 0]]) #11


permutation = [0, 2, 4, 6, 8, 10, 1, 3, 5, 7, 9, 11]
m1_perm = m1[permutation, :][:, permutation]
print("m1**2 = ", np.dot(m1, m1))
print("m1_perm**2 = ", np.dot(m1_perm, m1_perm))

m1**2 =  [[3 0 2 0 1 0 0 0 1 0 2 0]
 [0 3 0 2 0 1 0 0 0 1 0 2]
 [2 0 3 0 2 0 1 0 0 0 1 0]
 [0 2 0 3 0 2 0 1 0 0 0 1]
 [1 0 2 0 3 0 2 0 1 0 0 0]
 [0 1 0 2 0 3 0 2 0 1 0 0]
 [0 0 1 0 2 0 3 0 2 0 0 0]
 [0 0 0 1 0 2 0 3 0 2 0 1]
 [1 0 0 0 1 0 2 0 3 0 1 0]
 [0 0 0 0 0 1 0 2 0 2 0 1]
 [2 0 1 0 0 0 1 0 2 0 2 0]
 [0 2 0 1 0 0 0 1 0 2 0 3]]
m1_perm**2 =  [[3 2 1 0 1 2 0 0 0 0 0 0]
 [2 3 2 1 0 1 0 0 0 0 0 0]
 [1 2 3 2 1 0 0 0 0 0 0 0]
 [0 1 2 3 2 0 0 0 0 0 0 0]
 [1 0 1 2 3 1 0 0 0 0 0 0]
 [2 1 0 1 2 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 2 1 0 1 2]
 [0 0 0 0 0 0 2 3 2 1 0 1]
 [0 0 0 0 0 0 1 2 3 2 1 0]
 [0 0 0 0 0 0 0 1 2 3 2 1]
 [0 0 0 0 0 0 0 0 1 2 2 1]
 [0 0 0 0 0 0 2 1 0 1 2 3]]


## Block matrix generation

In [18]:

block = Graph.block_from_txt("datasets/block.txt")

p_flip = 0.1

for i in range(0, block.first_partition_size):
    for j in range(block.first_partition_size+1, block.adj_matrix.shape[1]):
        if block.adj_matrix[i, j] == 1 and np.random.rand() < p_flip:
                block.adj_matrix[i, j] = not block.adj_matrix[i, j]
                block.adj_matrix[j, i] = not block.adj_matrix[j, i]

        elif block.adj_matrix[i, j] == 0 and np.random.rand() < p_flip:
                block.adj_matrix[i, j] = not block.adj_matrix[i, j]
                block.adj_matrix[j, i] = not block.adj_matrix[j, i]

block.to_txt(f"datasets/block_{p_flip}.txt")




In [19]:

# for i in range(6):
#     block = generate_block_matrix(100, 10, i / 100)
#     graph = Graph.from_adj_matrix(block)
#     print("p_fip = ", i/100,"clustering coeff. = ", graph.global_clustering_coefficient())

NameError: name 'generate_block_matrix' is not defined

## Exploring the spectral approach

In [6]:
dataset = "flickr"
G = Graph.from_txt(f"datasets/{dataset}.txt")
G1, G2 = G.split_dataset(common_prop=0.0, graph1_prop=0.1)
A = np.dot(G.adj_matrix, G.adj_matrix.T)

### Sign assignment

In [8]:
def erdos(G_square):
    M_star = np.zeros_like(G_square)
    U_A, S_A, V_A = np.linalg.svd(G_square, compute_uv=True)
    for i in tqdm(range(S_A.shape[0])):

        S_Gi_plus = np.sqrt(S_A[i])
        S_Gi_minus =  - np.sqrt(S_A[i])

        M_star_plus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_plus * V_A[i, :].reshape(1, -1)
        M_star_minus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_minus * V_A[i, :].reshape(1, -1)

        binary_M_star_plus = np.where(M_star_plus > 0.5, 1, 0)
        binary_M_star_minus = np.where(M_star_minus > 0.5, 1, 0)

        distance_plus = np.linalg.norm(M_star_plus - binary_M_star_plus, ord='fro')
        distance_minus = np.linalg.norm(M_star_minus - binary_M_star_minus, ord='fro')

        if distance_plus < distance_minus:
            M_star = M_star_plus
        else:
            M_star = M_star_minus

    M_star = np.where(M_star > 0.5, 1, 0)
    return M_star


M_star = erdos(A)
print("Number of errors : ", np.linalg.norm(G.adj_matrix - M_star) ** 2)

100%|██████████| 7575/7575 [2:00:23<00:00,  1.05it/s]   


Number of errors :  24545.999999999996


### Revisited sign assignment

In [ ]:
def GRAND(A, Gstar, threshold):
    M_star = np.zeros_like(Gstar.adj_matrix)
    U_A, S_A, V_A = np.linalg.svd(A, compute_uv=True)

    Gstar_edges = np.argwhere(Gstar.adj_matrix == 1)
    number_of_edges = np.sum(np.diag(A))


    for i in tqdm(range(S_A.shape[0])):


        S_Gi_plus = np.sqrt(S_A[i])
        S_Gi_minus =  - np.sqrt(S_A[i])

        M_star_plus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_plus * V_A[i, :].reshape(1, -1)
        M_star_minus = M_star + U_A[:, i].reshape(-1, 1) * S_Gi_minus * V_A[i, :].reshape(1, -1)

        binary_M_star_plus = np.where(M_star_plus >= threshold, 1, 0)
        binary_M_star_minus = np.where(M_star_minus >= threshold, 1, 0)

        Gstar_edges = np.argwhere(Gstar.adj_matrix == 1)
        Gstar_non_edges = np.argwhere(Gstar.adj_matrix == 0)

        # compute distance only for the slots of Gstar that are 1
        # distance_plus = np.linalg.norm(M_star_plus - Gstar.adj_matrix) + np.linalg.norm(M_star_plus - binary_M_star_plus)
        # distance_minus = np.linalg.norm(M_star_minus - Gstar.adj_matrix) + np.linalg.norm(M_star_minus - binary_M_star_minus)

        distance_plus_M_G = np.linalg.norm(M_star_plus[Gstar_edges[:, 0], Gstar_edges[:, 1]] - Gstar.adj_matrix[Gstar_edges[:, 0], Gstar_edges[:, 1]])
        distance_minus_M_G = np.linalg.norm(M_star_minus[Gstar_edges[:, 0], Gstar_edges[:, 1]] - Gstar.adj_matrix[Gstar_edges[:, 0], Gstar_edges[:, 1]])

        distance_plus_M_G += np.linalg.norm(M_star_plus[Gstar_non_edges[:, 0], Gstar_non_edges[:, 1]] - Gstar.adj_matrix[Gstar_non_edges[:, 0], Gstar_non_edges[:, 1]])
        distance_minus_M_G += np.linalg.norm(M_star_minus[Gstar_non_edges[:, 0], Gstar_non_edges[:, 1]] - Gstar.adj_matrix[Gstar_non_edges[:, 0], Gstar_non_edges[:, 1]])

        distance_plus_M_binary = np.linalg.norm(M_star_plus - binary_M_star_plus, ord='fro')
        distance_minus_M_binary = np.linalg.norm(M_star_minus - binary_M_star_minus, ord='fro')

        # distance_plus_M_non_edges = np.abs(np.linalg.norm(binary_M_star_plus) - number_of_edges) /2
        # distance_minus_M_non_edges = np.abs(np.linalg.norm(binary_M_star_minus) - number_of_edges)/2
        # print("distance_plus_M_non_edges = ", distance_plus_M_non_edges)
        # print("distance_minus_M_non_edges = ", distance_minus_M_non_edges)


        distance_plus = distance_plus_M_G + distance_plus_M_binary #+ distance_plus_M_non_edges
        distance_minus = distance_minus_M_G + distance_minus_M_binary #+ distance_minus_M_non_edges

        if distance_plus < distance_minus:
            M_star = M_star_plus
        else:
            M_star = M_star_minus

    M_star = np.where(M_star >= threshold, 1, 0)

    for edge in Gstar_edges:
        M_star[edge[0], edge[1]] = 1

    return M_star

M_star = GRAND(A, G1, 0.5)
print("Number of errors : ", np.linalg.norm(G.adj_matrix - M_star) ** 2)
print("Number of edges : ", np.sum(M_star)/2)
print("Number of edges in G : ", np.sum(G.adj_matrix)/2)

In [11]:
import pandas as pd

dataset1 = pd.read_csv("datasets/josee.txt", sep="\t", header=None)
dataset1 

,0,1,2
0,1,2,1
1,1,4,1
2,1,12,1
3,2,1,1
4,2,3,1
5,2,11,1
6,3,2,1
7,3,4,1
8,3,6,1
9,4,1,1
